## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-14-06-53-26 +0000,nyt,"Crane Falls on Passenger Train in Thailand, Ki...",https://www.nytimes.com/2026/01/14/world/asia/...
1,2026-01-14-06-46-52 +0000,startribune,The latest: Tensions remain high in Minnesota ...,https://www.startribune.com/ice-raids-minnesot...
2,2026-01-14-06-46-22 +0000,nyt,Iran’s Leaders May Survive Protests. But Anger...,https://www.nytimes.com/2026/01/14/world/middl...
3,2026-01-14-06-43-56 +0000,wapo,Saks files for bankruptcy as it struggles to p...,https://www.washingtonpost.com/business/2026/0...
4,2026-01-14-06-32-49 +0000,nypost,"OnlyFans models arrested, kicked off American ...",https://nypost.com/2026/01/14/us-news/onlyfans...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2349/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
86,trump,54
19,iran,38
16,ice,21
127,china,17
400,new,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
178,2026-01-13-21-07-00 +0000,wsj,President Trump said the economy was poised fo...,https://www.wsj.com/politics/policy/trump-pred...,136
39,2026-01-14-03-31-29 +0000,nypost,Trump threatens ‘very strong action’ against I...,https://nypost.com/2026/01/13/us-news/trump-th...,126
314,2026-01-13-14-35-20 +0000,nyt,Trump Says He Will Impose Tariffs on Iran’s Tr...,https://www.nytimes.com/2026/01/13/world/middl...,119
22,2026-01-14-04-49-41 +0000,bbc,"More than 2,000 people reported killed at Iran...",https://www.bbc.com/news/articles/cr57vmznmd9o...,119
128,2026-01-13-22-43-33 +0000,nypost,The potential targets Trump could strike in Ir...,https://nypost.com/2026/01/13/world-news/the-p...,118


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
178,136,2026-01-13-21-07-00 +0000,wsj,President Trump said the economy was poised fo...,https://www.wsj.com/politics/policy/trump-pred...
43,78,2026-01-14-02-55-40 +0000,nypost,Mayor Zohran Mamdani finally condemns Iran reg...,https://nypost.com/2026/01/13/us-news/mayor-zo...
305,70,2026-01-13-15-06-00 +0000,wsj,The Justice Department’s investigation of Fede...,https://www.wsj.com/economy/central-banking/th...
99,69,2026-01-14-00-11-20 +0000,wapo,New York governor will push for state lawsuits...,https://www.washingtonpost.com/nation/2026/01/...
184,54,2026-01-13-20-51-30 +0000,cbc,"As Carney heads to China to talk trade, Ontari...",https://www.cbc.ca/news/canada/toronto/doug-fo...
221,54,2026-01-13-18-51-55 +0000,latimes,Supreme Court likely to uphold state bans on t...,https://www.latimes.com/politics/story/2026-01...
190,53,2026-01-13-20-29-53 +0000,wapo,Prosecutors seek death for South Korea’s forme...,https://www.washingtonpost.com/world/2026/01/1...
50,46,2026-01-14-02-21-31 +0000,nypost,BLM protesters hijack LA Police Commission mee...,https://nypost.com/2026/01/13/us-news/blm-prot...
134,44,2026-01-13-22-37-28 +0000,nypost,Eric Adams tells heckler ‘go f–k yourself’ as ...,https://nypost.com/2026/01/13/us-news/eric-ada...
11,42,2026-01-14-05-49-08 +0000,nypost,Saks Global files for bankruptcy after Neiman ...,https://nypost.com/2026/01/14/business/saks-fi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
